In [2]:
import networkx as nx
import simpy
import graphviz
import random

def merge_sort_traffic(data):
    if len(data) > 1:
        mid = len(data) // 2
        left_half = data[:mid]
        right_half = data[mid:]
        
        merge_sort_traffic(left_half)
        merge_sort_traffic(right_half)
        
        i = j = k = 0
        while i < len(left_half) and j < len(right_half):
            if left_half[i][1] > right_half[j][1]:
                data[k] = left_half[i]
                i += 1
            else:
                data[k] = right_half[j]
                j += 1
            k += 1
        
        while i < len(left_half):
            data[k] = left_half[i]
            i += 1
            k += 1
        
        while j < len(right_half):
            data[k] = right_half[j]
            j += 1
            k += 1

class TrafficLight:
    def __init__(self, env, location, cycle_time=60):
        self.env = env
        self.location = location
        self.cycle_time = cycle_time
        self.state = "RED"
        self.process = env.process(self.run())

    def run(self):
        while True:
            traffic_density = random.randint(1, 100)
            green_time = self.optimize_green_time(traffic_density)
            self.state = "GREEN"
            yield self.env.timeout(green_time)
            self.state = "RED"
            yield self.env.timeout(self.cycle_time - green_time)

    def optimize_green_time(self, traffic_density):
        return max(10, min(traffic_density // 2, 60))

class TrafficSimulation:
    def __init__(self):
        self.env = simpy.Environment()
        self.road_network = self.create_road_network()
        self.vehicles = []
        self.traffic_lights = {}
        self.setup_traffic_lights()
        self.env.process(self.run_simulation())

    def create_road_network(self):
        return {
            "Mumbai": {"Pune": 150, "Nashik": 180, "Hyderabad": 720},
            "Pune": {"Mumbai": 150, "Nashik": 200, "Bangalore": 840},
            "Nashik": {"Pune": 200, "Nagpur": 600, "Mumbai": 180},
            "Nagpur": {"Nashik": 600, "Hyderabad": 500},
            "Hyderabad": {"Nagpur": 500, "Bangalore": 570, "Mumbai": 720},
            "Bangalore": {"Pune": 840, "Hyderabad": 570}
        }

    def setup_traffic_lights(self):
        for location in self.road_network.keys():
            self.traffic_lights[location] = TrafficLight(self.env, location)

    def add_vehicle(self, vehicle_id, start, destination):
        route = self.dijkstra_shortest_path(start, destination)
        if not route:
            print(f"No route found for {vehicle_id} from {start} to {destination}")
            return
        vehicle = Vehicle(self.env, vehicle_id, route, self.road_network, self.traffic_lights)
        self.vehicles.append(vehicle)
        self.env.process(vehicle.drive())

    def run_simulation(self):
        while True:
            yield self.env.timeout(5)

    def run(self, simulation_time=2000):
        self.env.run(until=simulation_time)
        self.visualize_traffic()

    def visualize_traffic(self):
        dot = graphviz.Digraph()
        for node in self.road_network:
            dot.node(node)
        for node, neighbors in self.road_network.items():
            for neighbor, weight in neighbors.items():
                dot.edge(node, neighbor, label=str(weight))
        dot.render("traffic_network", format="png", view=True)

    def dijkstra_shortest_path(self, start, goal):
        shortest_paths = {start: (None, 0)}
        current_node = start
        visited = set()
        
        while current_node != goal:
            visited.add(current_node)
            destinations = self.road_network[current_node]
            weight_to_current_node = shortest_paths[current_node][1]
            
            for next_node, weight in destinations.items():
                weight += weight_to_current_node
                if next_node not in shortest_paths:
                    shortest_paths[next_node] = (current_node, weight)
                else:
                    current_shortest_weight = shortest_paths[next_node][1]
                    if current_shortest_weight > weight:
                        shortest_paths[next_node] = (current_node, weight)
            
            next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
            if not next_destinations:
                return None
            
            current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
        
        path = []
        while current_node is not None:
            path.append(current_node)
            next_node = shortest_paths[current_node][0]
            current_node = next_node
        return path[::-1]

class Vehicle:
    def __init__(self, env, vehicle_id, route, road_network, traffic_lights):
        self.env = env
        self.vehicle_id = vehicle_id
        self.route = route
        self.road_network = road_network
        self.traffic_lights = traffic_lights
        print(f"{self.vehicle_id} Route: {' -> '.join(self.route)}")

    def drive(self):
        total_time = 0
        for i in range(len(self.route) - 1):
            current_node = self.route[i]
            next_node = self.route[i + 1]
            travel_time = self.road_network[current_node][next_node]
            
            if self.traffic_lights[current_node].state == "RED":
                delay = random.randint(10, 60)
                print(f"{self.vehicle_id} waiting at red light in {current_node} for {delay}s")
                total_time+=delay
                yield self.env.timeout(delay)
            
            print(f"{self.vehicle_id} traveling from {current_node} to {next_node} (Time: {travel_time}s)")
            yield self.env.timeout(travel_time)
            total_time += travel_time
        
        print(f"{self.vehicle_id} reached destination {self.route[-1]} in {total_time}s")

if __name__ == "__main__":
    sim = TrafficSimulation()
    sim.add_vehicle("Truck1", "Mumbai", "Bangalore")
    sim.add_vehicle("Car1", "Pune", "Bangalore")
    sim.add_vehicle("Bus1", "Nashik", "Nagpur")
    sim.run(2000)


Truck1 Route: Mumbai -> Pune -> Bangalore
Car1 Route: Pune -> Bangalore
Bus1 Route: Nashik -> Nagpur
Truck1 traveling from Mumbai to Pune (Time: 150s)
Car1 traveling from Pune to Bangalore (Time: 840s)
Bus1 traveling from Nashik to Nagpur (Time: 600s)
Truck1 waiting at red light in Pune for 50s
Truck1 traveling from Pune to Bangalore (Time: 840s)
Bus1 reached destination Nagpur in 600s
Car1 reached destination Bangalore in 840s
Truck1 reached destination Bangalore in 1040s
